In [1]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

In [ ]:
# for ASTER data
# to a large degree, can probably just use the radiance data that they kick out. Does not take into effect
# atmospheric effects. Looking for differences in spectral properties observable from orbit
# okay if it is sort of coarse for the purpose of detecting change. 
# Mark is okay with this as long as you're using scene derived endmembers since the atmospheric inputs will be different
# on a scene to scene basis. 

# want to do another SMA type deal, using scene specific endmembers. Focus on VNIR and SWIR bands, ignore the thermal
# bands for now. Eventually, this could be useful for assessing surface melt as a result 
# start with early 2000's due to data quality issues

# Mark is going to give Charlie a paper on SMA from a geological perspective. \

# mark is bringing down drowns, multispectral and thermal. 3 drones!
# are there areas we want to specifically stake out specific areas.  

In [2]:
# Define the area of interest (AOI)
aoi = ee.Geometry.Rectangle([162.277817, -77.740157, 163.272100, -77.576571])

In [3]:
# Function to mask clouds using QA60 band
def maskClouds(image):
    qa = image.select('')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)

In [4]:
# Load ASTER image collection
aster_collection = ee.ImageCollection('ASTER/AST_L1T_003')

In [15]:
# Filter ASTER data by AOI and date range
filtered_aster = aster_collection.filterBounds(aoi)\
                                  .filterDate('2020-01-01', '2024-07-15')\
                                  .map(maskClouds)

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.map",
    "arguments": {
      "baseAlgorithm": {
        "functionDefinitionValue": {
          "argumentNames": [
            "_MAPPING_VAR_0_0"
          ],
          "body": {
            "functionInvocationValue": {
              "functionName": "Image.updateMask",
              "arguments": {
                "image": {
                  "argumentReference": "_MAPPING_VAR_0_0"
                },
                "mask": {
                  "functionInvocationValue": {
                    "functionName": "Image.and",
                    "arguments": {
                      "image1": {
                        "functionInvocationValue": {
                          "functionName": "Image.eq",
                          "arguments": {
                            "image1": {
                              "functionInvocationValue": {
                                "functionName": "Image.bitwiseAnd",
      

In [11]:
# Select thermal IR band (e.g., band 12)
thermal_band = 'B12'

# Function to select thermal IR band and rename
def selectThermalIR(image):
    return image.select(thermal_band).rename('Thermal_IR')

In [12]:
# Apply function to select thermal IR band to filtered ASTER collection
aster_with_thermal_ir = filtered_aster.map(selectThermalIR)

In [13]:
# Function to export image to Google Drive
def exportImageToDrive(image):
    image_name = image.id().getInfo().split('/')[-1]
    export_params = {
        'image': image,
        'description': image_name,
        'scale': 30,  # Adjust scale as needed
        'region': aoi,
        'fileFormat': 'GeoTIFF',
        'folder': 'EarthEngine'  # Specify your folder in Google Drive
    }
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()

In [14]:
# Export each image in the collection to Google Drive
aster_with_thermal_ir_list = aster_with_thermal_ir.toList(aster_with_thermal_ir.size())
for i in range(aster_with_thermal_ir.size().getInfo()):
    image = ee.Image(aster_with_thermal_ir_list.get(i))
    exportImageToDrive(image)

print("Export tasks started. Check Google Drive for export status.")

Export tasks started. Check Google Drive for export status.
